In [1]:
import json
import PyPDF2
from os import listdir

#Ham doi ten tu cac FileFail thanh File cu                    
def getFileFail(name, listfile):
    for i in listfile:
        if i[12:16] == name[4:8]: return i
    return False

#DS 616 file pdf ban dau
list_files = listdir(r'C:\Users\an\Desktop\New folder')
#DS cac file pdf khong co du lieu ban dau
list_filesN = listdir(r'C:\Users\an\Desktop\NewFile')


# File chua 420 file pdf lay lan 1
File = open('successFile1.txt').read().split('\n')[:-1]

# File chua 126 file pdf lay lan 2
FileFail = open('successFileFail.txt').read().split('\n')[:-1]
# File chua 126 file pdf lay lan 2 da dc doi ten
newFile = []
for i in FileFail:
    newFile.append(getFileFail(i, list_files))
    


In [2]:
#Ham lay text de kiem tra
def Text(path, num):
    a = r'C:\Users\an\Desktop\New folder\{}'.format(path)
    file = open(a, 'rb')
    fileReader = PyPDF2.PdfFileReader(file)
    page = fileReader.getPage(num)
    text = page.extractText().lower()
    newtext = ' '.join(text.strip().split('\n'))
    newtext1 = str(newtext.encode("ascii", "ignore"))
    return newtext1.strip()
#Ham lay text(str) va otext(list) de tach du lieu va ghep lai cau hoan chinh
def getText(path, num):
    bien = getTittle(path)
    text = Text(path, num)[bien[0]:bien[1]]
    oText = text.strip().split()
    newtext1= ''.join(oText)
    return oText, newtext1

### loai bo noi dung bi trung o dau va cuoi moi trang.
def getTittle(path):
    text1 = Text(path, 2)
    text2 = Text(path, 3)
    if (text1[0] == text2[0]) and (text1[-5:] == text2[-5:]):
        for i in range(300):
            if(text1[i] != text2[i]):
                bien1 = i
                break
        count = 1
        for i in range(-1,-300,-1):
            if(text1[i] != text2[i]):
                if(count == 0):
                    bien2 = i
                    return bien1, bien2+1
                else:
                    count-=1
    else:
        count = 1
        for i in range(250):
            if(text1[i] != text2[i]):
                if(count == 0):
                    return i, None
                else:
                    count-=1
                    
# tra ve mang chua text khong co khoang trang dung de dua ve text ban dau                    
def timMang(mang, text):
    for i in range(len(mang)):
        if len(''.join(mang[i:])) <= len(text):
            return mang[i:]
def timMangSau(mang, text):    
    for i in range(len(mang)):
        if len(''.join(mang[:i])) == len(text):
            return mang[:i]
        if len(''.join(mang[:i])) >= len(text):
            return mang[:i-1]


In [3]:
#lay muc luc
def getTableOfContentPage(path):
    a = r'C:\Users\dat\Desktop\New folder\{}'.format(path)
    file = open(a, 'rb')
    fileReader = PyPDF2.PdfFileReader(file)
    pages = fileReader.getNumPages()
    for i in range(pages):
        page = fileReader.getPage(i)
        text = page.extractText().lower()
        newtext = ' '.join(text.strip().split('\n'))
        newtext1= ''.join(text.strip().split())
        newstr = str(newtext1.encode("ascii", errors='ignore'))
#         if "b'efsascientificreport'" in newstr or "b''" in newstr or "b'pd'" in newstr:
#             return 1
        if 'tableofcontents' in newstr:
            return i
    return False
#Lay trang Thuoc Tinh 2 (TT2)
def getImpactHumanPage(path):
    a = r'C:\Users\dat\Desktop\New folder\{}'.format(path)
    file = open(a, 'rb')
    fileReader = PyPDF2.PdfFileReader(file)
    pages = fileReader.getNumPages()
    for i in range(pages):
        page = fileReader.getPage(i)
        text = page.extractText().lower()
        newtext = ' '.join(text.strip().split('\n'))
        newtext1= ''.join(text.strip().split())
        newstr = str(newtext1.encode("ascii", errors='ignore'))                
        if 'impactonhumanandanimalhealth' in newstr or 'rateandextentofabsorption' in newstr:
            return i
    return False

#Lay noi dung trang muc luc
def getTextTOC(path, num):      
    oText1, text1 = getText(path, num)
    oText2, text2 = getText(path, num+1)
    if text2.startswith("background") or text2.startswith("ackground") or text2.startswith("ckground") or text2.startswith("commissionregulation") or text2.startswith('4efsajournal2011;9(7):2300') or text2.startswith('4efsajournal2011;9(7):2299'):
        return text1
    return text1 + text2

def checknum(text):
    try:
        return int(text[-4:-2])
    except:
        try:
            return int(text[-3:-1])
        except:
            try:
                return int(text[-2:])
            except:
                try:
                    return int(text[-1:])
                except:
                    return False
#Lay so trang chua TT1
def getPageTT1(text):
    keys1 = ['appendix2', 'appendixb', 'appendixii']

    for i in keys1:
        array = text.split(i,1)
        if len(array) == 2:
            return checknum(array[0].strip())
    keys2 = ['listofabbreviations']
    for i in keys2:
        array = text.split(i,1)
        if len(array) ==2:
            return checknum(array[0].strip())
    if 'abbreviations' in text and 'appendix' in text:
        array = text.split('appendixa')
        return checknum(array[0].strip())
                                                                                                         
    keys3 = ['laboratorydoseresponsetest',
             'residuesrequiringfurtherassessmen',
             'preliminaryscreeningdat',"'efsajournal2013;11(1):3066",
             'worst-caseuse(amongtheeurepresentativeuses):pomefruit(6x0.100kga.s./ha)','monitoring/enforcementmethod']
    for i in keys3:
        array = text.split(i,1)
        if len(array) ==2:
            return checknum(array[0].strip())
    array = text.split('abbreviations')
    if len(array) ==2 :
        return checknum(array[0].strip())
    return checknum(text.strip())

#Lay trang chua background1 (trang bat dau noi dung background)
def getPageBackground1(text):
    keys1 = ['theactivesubstanceandtheformulatedproduct',
            'theidentityofthemicro-organismandthepropertiesoftheformulatedproduct',
            'theidentityofthemicroorganismandthepropertiesoftheformulatedproduct',
            'theidentityofthemicroorganismsandthepropertiesoftheformulatedproducts',
            'conclusionsoftheevaluation']
    for i in keys1:
        array = text.split(i,1)
        if len(array) == 2:
            return checknum(array[0].strip())
    return False
#Lay trang chua background2 (trang ket thuc)
def getPageBackground2(text):
    keys1 = ['specificconclusionsoftheevaluation',
             '1.identity,physical/chemical/technicalpropertiesandmethodsofanalysis', 
             'conclusionsoftheevaluation','datagaps']
    for i in keys1:
        array = text.split(i,1)
        if len(array) == 2:
            return checknum(array[0].strip())
    return False


In [22]:
#Lay trang muc luc
dic = dict()
for i in list_files:
    page = getTableOfContentPage(i)
    if page != False:
        dic[i] = page    

#Lay noi dung trang muc luc 
ContentOFTOC = dict()
for i in dic:
    ContentOFTOC[i] = getTextTOC(i, dic[i])

with open('textTOC.txt', 'w') as convert_file:
     convert_file.write(json.dumps(ContentOFTOC))

In [193]:
f = open('textTOC.txt')
textTOC = json.load(f)

newdic1 = dict()
nonList = []
nonConclu = []
for  i in File+newFile:
    if 'listofstudiestobegenerated' in textTOC[i]:
        bien = textTOC[i].split('listofstudiestobegenerated',1)
        if len(bien) == 2:
            newdic1[i] = bien[1]
        if 'conclusionsandrecommendations' not in textTOC[i]: nonConclu.append(i)
    elif 'conclusionsandrecommendations' in textTOC[i]:
        nonList.append(i)
        bien = textTOC[i].split('conclusionsandrecommendations',1)
        if len(bien) == 2:
            newdic1[i] = bien[1]
    else:
        nonList.append(i)
        nonConclu.append(i)
        bien = textTOC[i].split('concerns',1)
        if len(bien) == 2:
            newdic1[i] = bien[1]

for i in newdic1:
    if 'references' in newdic1[i] and 'appendices' in newdic1[i]:
        bien = newdic1[i].split('appendi',1)
        if len(bien) == 2:
            newdic1[i] = bien[0]
    elif 'references' in newdic1[i] and 'abbreviations' in newdic1[i]:
        bien = newdic1[i].split('abbreviations',1)
        if len(bien) == 2:
            newdic1[i] = bien[0]
    else:
        bien = newdic1[i].split('appendi',2)
        if len(bien) == 3:
            newdic1[i] = bien[0] + bien[1]
        else:
            bien = newdic1[i].split('abbreviations',1)
            if len(bien) == 2:
                newdic1[i] = bien[0]
page1 = dict()
for i in newdic1:
    bien = newdic1[i].split('conclusionsandrecommendations', 1)
    if len(bien) == 2:
        page1[i] = checknum(bien[0])
        nonConclu.append(i)
    else:
        bien = newdic1[i].split('particularconditionsproposedtobetakenintoaccount', 1)
        if len(bien) == 2:
            page1[i] = checknum(bien[0])
            nonConclu.append(i)
        else:
            bien = newdic1[i].split('1.issuesthatcouldnotbefinalised', 1)
            if len(bien) == 2:
                check = checknum(bien[0])
                if type(check) != int:
                    bien = newdic1[i].split('9.1.concernsfortherepresentativeusesevaluated', 1)
                    check = checknum(bien[0])
                    page1[i] = check
                    nonConclu.append(i)
                else:    
                    page1[i] = checknum(bien[0])
                    nonConclu.append(i)
            else:
                bien = newdic1[i].split('3.concerns', 1)
                if len(bien) == 2:
                    check = checknum(bien[0])
                    if type(check) != int: print(newdic1[i])
                    page1[i] = checknum(bien[0])
                    nonConclu.append(i)
                else:
                    bien = newdic1[i].split('issuesthatcouldnotbefinalised', 1)
                    if len(bien) == 2:
                        check = checknum(bien[0])
                        if type(check) != int: print(newdic1[i])
                        page1[i] = check
                        nonConclu.append(i)
                    else:                           
                        ######################################################################################################
                        bien = newdic1[i].split('1.issuesthatcouldnotbenalised', 1)
                        if len(bien) == 2:
                            check = checknum(bien[0])
                            if type(check) != int: 
                                bien = newdic1[i].split('9.1.concernsfortherepresentativeusesevaluated', 1)
                                check = checknum(bien[0])
                                page1[i] = check
                                nonConclu.append(i)
                            else:
                                page1[i] = check
                                nonConclu.append(i)
                        else:
                            bien = newdic1[i].split('9.1issuesthatcouldnotbenalised', 1)
                            if len(bien) == 2:
                                check = checknum(bien[0])
                                if type(check) != int: print(bien[0])
                                page1[i] = check
                                nonConclu.append(i)
                            else:
                                bien = newdic1[i].split('10.1.concernsfortherepresentativeusesevaluated', 1)
                                if len(bien) == 2:
                                    check = checknum(bien[0])
                                    if type(check) != int: print(bien[0])
                                    page1[i] = check
                                    nonConclu.append(i)
                                else:    
                                    print(i)
                                    print(newdic1[i])
                                    print('\n')
                                    
page2 = dict()
for i in newdic1:
    check = checknum(newdic1[i])
    if type(check) != int:
        text = newdic1[i].split('9.3.')[0]
        check = checknum(text)
        if type(check) == int:
            page2[i] = check        
        else:
            check = checknum(newdic1[i].split('appendixb')[0])
            page2[i] = check 
    else:
        page2[i] = check
page1['j.efsa.2020.6006.pdf'] = 10    

newTT = dict()
onewTT = dict()

for i in newdic1:
    text = ''
    otext = []
    for j in range(page1[i]-1, page2[i]-1):
        bien = getText(i, j)
        text+= bien[1]
        otext+= bien[0]
    newTT[i] = text
    onewTT[i] = otext
    
# with open('newTT.txt', 'w') as convert_file:
#      convert_file.write(json.dumps(bien))
# with open('onewTT.txt', 'w') as convert_file:
#      convert_file.write(json.dumps(obien))

listList = []
listConclu = []
listConcern = []

f = open('newTT.txt')
bien = json.load(f)
f1 = open('onewTT.txt')
obien = json.load(f1)

for i in File + newFile:
    if i in nonConclu:
        if i in nonList:
            listList.append(None)
            listConclu.append(None)
            listConcern.append(' '.join(obien[i]))
        else:
            array = bien[i].split('criticalareasofconcern', 1)
            if len(array) == 2:
                listConclu.append(None)
                listList.append(' '.join(timMangSau(obien[i], array[0])))
                listConcern.append(' '.join(timMang(obien[i], array[1])))
            else:
                array = bien[i].split('concerns', 1)
                if len(array) == 2:
                    listConclu.append(None)
                    listList.append(' '.join(timMangSau(obien[i], array[0])))
                    listConcern.append(' '.join(timMang(obien[i], array[1])))
                else:
                    if i == 'j.efsa.2010.1526.pdf':
                        array = bien[i].split("uses.'the")
                        listConclu.append(None)
                        listList.append(' '.join(timMangSau(obien[i], array[0])) + 'uses.')
                        listConcern.append("the" +' '.join(timMang(obien[i], array[1])))
                    if i == 'j.efsa.2010.1708.pdf':
                        array = bien[i].split("operations.")
                        listConclu.append(None)
                        listList.append(' '.join(timMangSau(obien[i], array[0])) + 'operations.')
                        listConcern.append(' '.join(timMang(obien[i], array[1])))
    else:
        if i in nonList:
            print(i)
        else:
            array = bien[i].split('criticalareasofconcern', 1)
            if len(array) == 2:
                listConcern.append(' '.join(timMang(obien[i], array[1])))                
                text = array[0]
                otext = timMangSau(obien[i], array[0])
                array = text.split('conclusionsandrecommendations')
                if len(array) == 2:
                    listConclu.append(' '.join(timMang(otext, array[1])))
                    listList.append(' '.join(timMangSau(otext, array[0])))
                else:
                    array = text.split('onclusionsandrecommendations')
                    if len(array) == 2:
                        listConclu.append(' '.join(timMang(otext, array[1])))
                        listList.append(' '.join(timMangSau(otext, array[0])))
                    else:
                        array = text.split('onthebasis')
                        listConclu.append("the conclusion was reached on the basis" + ' '.join(timMang(otext, array[1])))
                        listList.append(' '.join(timMangSau(otext, array[0])))
            else:
                array = bien[i].split('riticalareasofconcern', 1)
                if len(array) == 2:
                    if i == 'j.efsa.2005.102r.pdf':
                        listConcern.append(' '.join(timMang(obien[i], array[1])).split('8. 8')[0])
                    else:
                        listConcern.append(' '.join(timMang(obien[i], array[1])))
                    text = array[0]
                    otext = timMangSau(obien[i], array[0])
                    array = text.split('conclusionsandrecommendations')
                    if len(array) == 2:
                        listConclu.append(' '.join(timMang(otext, array[1])))
                        listList.append(' '.join(timMangSau(otext, array[0])))
                    else:
                        array = text.split('onclusionsandrecommendations')
                        if len(array) == 2:
                            listConclu.append(' '.join(timMang(otext, array[1])))
                            listList.append(' '.join(timMangSau(otext, array[0])))
                else:
                    listConcern.append(None)
                    array = bien[i].split('conclusionsandrecommendations')
                    if len(array) == 2:
                        if i == 'j.efsa.2007.103r.pdf':
                            listConclu.append(' '.join(timMang(obien[i], array[1])).split('critical areas of conc')[0])
                            listList.append(' '.join(timMangSau(obien[i], array[0])))
                        else:
                            listConclu.append(' '.join(timMang(obien[i], array[1])))
                            listList.append(' '.join(timMangSau(obien[i], array[0])))
import pandas as pd
xong = {
    'ten': File+newFile,
    'tt1': listList,
    'tt2': listConclu,
    'tt3': listConcern
}

df = pd.DataFrame(list(zip(File+newFile, listList, listConclu, listConcern))
                           , columns=['Name', 'List of studies to be generated', 'Conclusions and Recommendations', 'Concerns'])
df.to_csv('data10-8.csv', index=False)

In [141]:
#Lay so trang của Data can tim
f = open('textTOC.txt')
textTOC = json.load(f)

pageTT1 = dict()

for i in File:
    check = getPageTT1(textTOC[i])
    if check != False:
        pageTT1[i] = check

# Get content of TT1

In [6]:
#Lấy text chứa TT1
# Nếu dùng lấy dữ liệu trong list File thì lấy trang từ list pageTT1

ContentOfTT1 = dict()
oContentOfTT1 = dict()
for i in File:
    try:
        trang1 = getText(i, pageTT1[i]-1)
        trang2 = getText(i, pageTT1[i])
        trang3  = getText(i, pageTT1[i]+1)
    except:
        print(i)
    if 'molecularformula' in trang3[1] or 'identityofrelevantimpurities' in trang3[1]:
        oContentOfTT1[i] =  trang1[0] + trang2[0] + trang3[0]
        ContentOfTT1[i] =  trang1[1] + trang2[1] + trang3[1]

    elif 'molecularformula' in trang2[1] or 'identityofrelevantimpurities' in trang2[1]:
        oContentOfTT1[i] =  trang1[0] + trang2[0]
        ContentOfTT1[i] =  trang1[1] + trang2[1]
    else:
        oContentOfTT1[i] = trang1[0]
        ContentOfTT1[i] = trang1[1]

with open('textTT1.txt', 'w') as convert_file:
     convert_file.write(json.dumps(ContentOfTT1))
with open('otextTT1.txt', 'w') as convert_file:
     convert_file.write(json.dumps(oContentOfTT1))

# Tách text chứa TT1 thành 2 phần TT1_1 và TT1_2 

In [8]:
def tach1(text):
    keys = ['identity(annexiia,point1)','chemicalname(iupac)','chemicalname']
    for i in keys:
        array = text.split(i,1)
        if len(array) == 2:
            return array
    return False

f = open('textTT1.txt')
textTT1 = json.load(f)

f1 = open('otextTT1.txt')
otextTT1 = json.load(f1)


textTT1_1 = dict()
otextTT1_1 = dict()

textTT1_2 = dict()
otextTT1_2 = dict()

for i in File:
    if i == 'j.efsa.2012.2520.pdf':
        array = textTT1[i].split('chemicalname(iupac)',1)
        textTT1_1[i] = array[0]
        otextTT1_1[i] = timMangSau(otextTT1[i], array[0])
        
        textTT1_2[i] = array[1]
        otextTT1_2[i] = timMang(otextTT1[i], array[1])

        continue
    check = tach1(textTT1[i])
    if check == False:
        textTT1_1[i] = textTT1[i]
        otextTT1_1[i] = otextTT1[i]
        
        textTT1_2[i] = None
        otextTT1_2[i] = None

    else:
        textTT1_1[i] = check[0]
        otextTT1_1[i] = timMangSau(otextTT1[i], check[0])
        
        textTT1_2[i] = check[1]
        otextTT1_2[i] = timMang(otextTT1[i], check[1])
        
with open('textTT1_1.txt', 'w') as convert_file:
     convert_file.write(json.dumps(textTT1_1))
with open('otextTT1_1.txt', 'w') as convert_file:
     convert_file.write(json.dumps(otextTT1_1))

with open('textTT1_2.txt', 'w') as convert_file:
     convert_file.write(json.dumps(textTT1_2))
with open('otextTT1_2.txt', 'w') as convert_file:
     convert_file.write(json.dumps(otextTT1_2))

# Lấy text chứa TT2

In [11]:
# Lay text chua nhom Thuoc Tinh 2 (TT2)

textTT2 = dict()
otextTT2 = dict()

for i in File:
    check = getImpactHumanPage(i)
    if check == False:
        textTT2[i] = None
        otextTT2[i] = None
    else:
        bien1 = getText(i,check)
        bien2 = getText(i,check+1)
#         bien3 = getText(i,check+2)
#         if 'ld50' in bien3[1] or 'lc50' in bien3[1] or 'eyeirritation' in bien3[1]:
#             otextTT2[i] = bien1[0] + bien2[0] + bien3[0]
#             textTT2[i] = bien1[1] + bien2[1] + bien3[1]
        if 'ld50' in bien2[1] or 'lc50' in bien2[1] or 'eyeirritation' in bien2[1]:
            otextTT2[i] = bien1[0] + bien2[0]
            textTT2[i] = bien1[1] + bien2[1]
        else:
            otextTT2[i] = bien1[0]
            textTT2[i] = bien1[1]

with open('textTT2.txt', 'w') as convert_file:
     convert_file.write(json.dumps(textTT2))
with open('otextTT2.txt', 'w') as convert_file:
    convert_file.write(json.dumps(otextTT2))

In [4]:
HMPage = dict()
fail = []
for i in File:
    check = getImpactHumanPage(i)
    if check == False:
        fail.append(i)
    else:
        HMPage[i] = check
MAPage = dict()
for i in HMPage:
    for j in range(HMPage[i]-1, HMPage[i]-6,-1):
        text = getText(i, j)[1]
        keys = ['technicala.s.(analyticaltechnique)','technicalas(principleofmethod)', 'technicalas(analyticaltechnique)', 'technicala.s.(principleofmethod)','technicalas(principleofthemethod)']
        for k in keys:
            if k in text:
                MAPage[i] = j
                break
            
for i in HMPage:
    if i not in MAPage:
        for j in range(HMPage[i], 0,-1):
            text = getText(i, j)[1]
            if 'bodyfluidsandtissues' in text:
                bien = j
            if 'methodsofanalysis' in text or 'analyticalmethodsfortheactivesubstance' in text or 'technicala.s.' in text or 'technicalas' in text:
                MAPage[i] = j
                HMPage[i] = bien+1
                break
                
for i in HMPage:
    if i not in MAPage: fail.append(i)
        
for i in MAPage:
    if HMPage[i] - MAPage[i] > 5:
        fail.append(i)
        

In [7]:
for i in MAPage:
    if HMPage[i] == MAPage[i]: 
        print(i in fail)

False


In [8]:
text = dict()
otext = dict()
for i in File:
    bien1 = ''
    bien2 = []
    if i in fail:
        text[i] = None
        otext[i] = None
    else:
        if HMPage[i] == MAPage[i]: 
            text[i] = None
            otext[i] = None
        else:
            for j in range(MAPage[i], HMPage[i]):
                trang = getText(i, j)
                bien1+=trang[1]
                bien2+=trang[0]
            text[i] = bien1
            otext[i] = bien2

In [29]:
len(fail)

40

In [9]:
with open('12-8text.txt', 'w') as convert_file:
     convert_file.write(json.dumps(text))
with open('12-8otext.txt', 'w') as convert_file:
     convert_file.write(json.dumps(otext))

# Lấy text Summary của 420 file đầu và 126 file sau

In [13]:
#GetSumMary tu trang dau den trang muc luc

summary = dict()
osummary = dict()
for i in File:
    text = ''
    mang = []
#biến dic là dict() chứa trang mục lục (int)
    for j in range(dic[i]):
        bien1 = getText(i, j)
        text += bien1[1]
        mang += bien1[0]
    summary[i] = text
    osummary[i] = mang

with open('summary.txt', 'w') as convert_file:
     convert_file.write(json.dumps(summary))
with open('osummary.txt', 'w') as convert_file:
     convert_file.write(json.dumps(osummary))

summaryFail = dict()
osummaryFail = dict()
for i in newFile:
    text = ''
    mang = []
    for j in range(dic[i]):
        bien1 = getText(i, j)
        text += bien1[1]
        mang += bien1[0]
    summaryFail[i] = text
    osummaryFail[i] = mang

# with open('summaryFail.txt', 'w') as convert_file:
#      convert_file.write(json.dumps(summaryFail))
# with open('osummaryFail.txt', 'w') as convert_file:
#      convert_file.write(json.dumps(osummaryFail))

# Lấy text Background

In [28]:
############################################################## lấy background 420 file đầu

pageBackground1 = dict()
for i in File:
    text = textTOC[i]  
    bien = getPageBackground1(text)
    if bien != False:
        pageBackground1[i] = bien
        
pageBackground2 = dict()
for i in File:
    text = textTOC[i]  
    bien = getPageBackground2(text)
    if bien != False:
        pageBackground2[i] = bien
        
background = dict()
obackground = dict()

for i in File:
    text = ''
    mang = []
    for j in range(pageBackground1[i]-1, pageBackground2[i]):
        try:
            bien = getText(i, j)
        except:
            print(i, j)
            continue
        text += bien[1]
        mang += bien[0]
    background[i] = text
    obackground[i] = mang
    
with open('background.txt', 'w') as convert_file:
     convert_file.write(json.dumps(background))
with open('obackground.txt', 'w') as convert_file:
     convert_file.write(json.dumps(obackground))

######################################################################### 126 file sau
        
pageBackground2Fail = dict()
for i in newFile:
    text = textTOC[i]  
    bien = getPageBackground2(text)
    if bien != False:
        pageBackground2Fail[i] = bien
        
backgroundFail = dict()
obackgroundFail = dict()

for i in newFile:
    text = ''
    mang = []
    for j in range(dic[i]+1, pageBackground2Fail[i]):
        try:
            bien = getText(i, j)
        except:
            print(bien, j)
        text += bien[1]
        mang += bien[0]
    backgroundFail[i] = text
    obackgroundFail[i] = mang
    
with open('backgroundFail.txt', 'w') as convert_file:
     convert_file.write(json.dumps(backgroundFail))
with open('obackgroundFail.txt', 'w') as convert_file:
     convert_file.write(json.dumps(obackgroundFail))

In [ ]:
# TTCuoi = []

# def getTTcuoi(path, num):
#     text = ''
#     for i in range(num):
#         text += getText(path, i)[1]
#     if 'concernsarenotidenti' in text:
#         return 'Concerns are not identified'            
#     elif 'concernsareidenti' in text:
#         return 'Concerns are identified'

# for i in File+newFile:
#     check = getTTcuoi(i, dic[i])
#     TTCuoi.append(check)